In [1]:
using SeisIO, SeisNoise, JLD2, Dates, Plots, Interpolations
using SeisMonitoring: seisdvv_mwcs, seisdvv_stretching

In [2]:
# read reference stacked trace processed in Cascadia

t = jldopen("./reference_BP.EADB-BP.LCCB-11.jld2", "r")
Cref = t["2010-01-01T00:00:00--2022-06-01T00:00:00/0.9-1.2"]

CorrData with 1 Corrs
      NAME: "BP.EADB-BP.LCCB-11"               
        ID: "2010-01-01"                       
       LOC: 35.8952 N, -120.423 E, 224.0 m
      COMP: "11"                               
   ROTATED: false                              
 CORR_TYPE: "CC"                               
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.9
   FREQMAX: 1.2
    CC_LEN: 3600.0
   CC_STEP: 1800.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 1z, 1p
      MISC: 17 entries                         
     NOTES: 13 entries                         
      DIST: 12.528
       AZI: 318.74
       BAZ: 138.686
    MAXLAG: 100.0
         T: 2010-01-01T00:30:00                …
      CORR: 4001×1 Matrix{Float32}             


# 1. Case with no clock shift

In [3]:
# Workflow
# 1. Stretch the trace from -0.1% to 0.1%
# 2. Store it in corrdata
# 3. Measure dv/v using seismeasurement, and see the quality of the evaluation.

In [4]:
dvmin = -0.001
dvmax = 0.001
ntrial = 11

tvec = -Cref.maxlag:(1/Cref.fs):Cref.maxlag
ϵ = range(dvmin, stop=dvmax, length=ntrial)
L = 1. .- ϵ # from velocity decrease to increase
tau = tvec * L'

Cstretched = deepcopy(Cref)
Cstretched.corr = zeros(Float32, size(Cref.corr)[1], ntrial)

# set of stretched/compressed current waveforms
waveform_ref = Cref.corr
for ii = 1:ntrial
	s = LinearInterpolation(tau[:,ii],vec(waveform_ref),extrapolation_bc=Flat())(tvec)
	Cstretched.corr[:, ii] = s
end

ref = vec(Cref.corr)

# Measure dv/v using seismeasurement

dvv_stretch = zeros(ntrial)
dvv_mwcs = zeros(ntrial)
dvv_mwcs0 = zeros(ntrial)

InputDict = Dict()
InputDict["mwcs_window_length"] = 6.0
InputDict["mwcs_window_step"] = 3.0
InputDict["mwcs_max_dt"] = 1.0
InputDict["mwcs_smoothing_half_win"] = 5
InputDict["coda_init_factor"] = 2
InputDict["max_coda_length"] = 40
InputDict["min_ballistic_twin"],  =  5.0
InputDict["background_vel"] = 1000.0

1000.0

In [5]:
for ii = 1:ntrial
    # stretchning
    MeasurementDict_stretch = seisdvv_stretching(ref, Cstretched.corr[:, ii], Cref.misc["timelag"], Cref.misc["coda_window"],
    Cref.freqmin,Cref.freqmax,
    dvmin=-0.02,　dvmax=0.02,
    ntrial_v=101)
    dvv_stretch[ii] = 1e-2*MeasurementDict_stretch["dvv_ts"]

    # MWCS
    MeasurementDict_mwcs = seisdvv_mwcs(ref, Cstretched.corr[:, ii],Cref.freqmin,Cref.freqmax,Cref.fs,-Cref.maxlag,
    InputDict["mwcs_window_length"], InputDict["mwcs_window_step"], InputDict["mwcs_max_dt"],
    InputDict["mwcs_smoothing_half_win"], InputDict["coda_init_factor"], InputDict["max_coda_length"],
    InputDict["min_ballistic_twin"], Cref.dist*1e3, InputDict["background_vel"])

    if isempty(MeasurementDict_mwcs)
        dvv_mwcs[ii]  = 0.0
        dvv_mwcs0[ii] = 0.0
        else
        dvv_mwcs[ii]  = -MeasurementDict_mwcs["dvv_mwcs"]
        dvv_mwcs0[ii] = -MeasurementDict_mwcs["dvv0_mwcs"]
        end
end

In [6]:
# Plot result

p1 = plot(tvec, Cref.corr, lc = "black", label=Cref.name*" "*"$(Cref.freqmin)-$(Cref.freqmax)Hz")
p1 = xlims!(-60, 60)
p1 = plot!(legend = :topright, framestyle = :box)

xaxis!("Lag time [s]")
yaxis!("CC")

p2 = plot([-0.002, 0.002], [-0.002, 0.002], ls=:dot, lc="black", labels="")
p2 = scatter!(collect(ϵ), dvv_mwcs, labels="mwcs no origin", ms=10, mc="blue")
p2 = scatter!(collect(ϵ), dvv_mwcs0,  labels="mwcs through origin", ms=7)
p2 = scatter!(collect(ϵ), dvv_stretch, labels="stretching", ms=7, mc="red")

xlims!(-0.0012, 0.0012)
ylims!(-0.0012, 0.0012)
xaxis!("dv/v synthetic true")
yaxis!("dv/v seismeasurement")
p2 = plot!(legend = :bottomright, framestyle = :box, aspect_ratio=1)
p2 = plot!(size=(700,600))

p = plot(p1, p2, layout = grid(2, 1, heights=[0.3, 0.7]))
savefig(p, "./seismeasurement_validation_$(Cref.name)_$(Cref.freqmin)-$(Cref.freqmax)Hz.png")

"/Users/kokubo/Dropbox/NIED_RESEARCH/new_git_SeisMonitoring_Paper/dev_new_git_v2_SeisMonitoring_Paper/Others/validation_of_SeisMeasurement/seismeasurement_validation_BP.EADB-BP.LCCB-11_0.9-1.2Hz.png"

# 2. Case with clock shift

The cross-correlation function of u1 and u2 is written as

$$ \psi(t) = \int_{-T}^{T} u_1(\tau) u_2(t-\tau) d\tau. $$

Given the clock shift D [sec] occurs in $u_2$, the CC is shifted as

$$ \psi'(t) = \int_{-T}^{T} u_1(\tau) u_2\{(t-D)-\tau \} d\tau. $$

Therefore, $\psi'(t) = \psi(t-D) $, indicating the CC is temporally shifted with D. 

Here, we mimic this effect and measure dv/v using the stretching method and mwcs with and without crossing zero.

In [7]:
clockshift = -30e-3 #2e-3 #[s]

# set of stretched/compressed current waveforms
waveform_ref = Cref.corr
# demonstrate clock shift with no stretch
ii = 6 # no stretch
s = LinearInterpolation(tau[:,ii] .- clockshift, vec(waveform_ref),extrapolation_bc=Flat())(tvec)

4001-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [8]:
p1 = plot(tvec, Cref.corr, lc = "black", title=Cref.name*" "*"$(Cref.freqmin)-$(Cref.freqmax)Hz", label="Reference")
p1 = plot!(tvec, s, lc = "blue", label="Time shifted with $(clockshift*1e3)ms")
p1 = xlims!(-60, 60)
p1 = plot!(legend = :best, framestyle = :box)

p2 = plot(tvec, Cref.corr, lc = "black", label="")
p2 = plot!(tvec, s, lc = "blue", label="Time shifted with $(clockshift*1e3)ms")
p2 = xlims!(0, 5)
p2 = plot!(legend = :topright, framestyle = :box)

p = plot(p1, p2, layout = grid(2, 1, heights=[0.5, 0.5]))
savefig(p, "./seismeasurement_clockshift_demo_$(Cref.name)_$(Cref.freqmin)-$(Cref.freqmax)Hz.png")

"/Users/kokubo/Dropbox/NIED_RESEARCH/new_git_SeisMonitoring_Paper/dev_new_git_v2_SeisMonitoring_Paper/Others/validation_of_SeisMeasurement/seismeasurement_clockshift_demo_BP.EADB-BP.LCCB-11_0.9-1.2Hz.png"

In [9]:
Cstretche_clockshift = deepcopy(Cref)
Cstretche_clockshift.corr = zeros(Float32, size(Cref.corr)[1], ntrial)

# set of stretched/compressed current waveforms
waveform_ref = Cref.corr
for ii = 1:ntrial
    s = LinearInterpolation(tau[:,ii] .- clockshift, vec(waveform_ref),extrapolation_bc=Flat())(tvec)
    Cstretche_clockshift.corr[:, ii] = s
end

ref = vec(Cref.corr)

# Measure dv/v using seismeasurement

dvv_stretch = zeros(ntrial)
dvv_mwcs = zeros(ntrial)
dvv_mwcs0 = zeros(ntrial)

InputDict = Dict()
InputDict["mwcs_window_length"] = 6.0
InputDict["mwcs_window_step"] = 3.0
InputDict["mwcs_max_dt"] = 1.0
InputDict["mwcs_smoothing_half_win"] = 5
InputDict["coda_init_factor"] = 2
InputDict["max_coda_length"] = 40
InputDict["min_ballistic_twin"],  =  5.0
InputDict["background_vel"] = 1000.0

1000.0

In [10]:
for ii = 1:ntrial
    # stretchning
    MeasurementDict_stretch = seisdvv_stretching(ref, Cstretche_clockshift.corr[:, ii], Cref.misc["timelag"], Cref.misc["coda_window"],
    Cref.freqmin,Cref.freqmax,
    dvmin=-0.02,　dvmax=0.02,
    ntrial_v=101)
    dvv_stretch[ii] = 1e-2*MeasurementDict_stretch["dvv_ts"]

    # MWCS
    MeasurementDict_mwcs = seisdvv_mwcs(ref, Cstretche_clockshift.corr[:, ii],Cref.freqmin,Cref.freqmax,Cref.fs,-Cref.maxlag,
    InputDict["mwcs_window_length"], InputDict["mwcs_window_step"], InputDict["mwcs_max_dt"],
    InputDict["mwcs_smoothing_half_win"], InputDict["coda_init_factor"], InputDict["max_coda_length"],
    InputDict["min_ballistic_twin"], Cref.dist*1e3, InputDict["background_vel"])

    if isempty(MeasurementDict_mwcs)
        dvv_mwcs[ii]  = 0.0
        dvv_mwcs0[ii] = 0.0
        else
        dvv_mwcs[ii]  = -MeasurementDict_mwcs["dvv_mwcs"]
        dvv_mwcs0[ii] = -MeasurementDict_mwcs["dvv0_mwcs"]
        end
end

In [11]:
# Plot result
p1 = plot(tvec, Cref.corr, lc = "black", label=Cref.name*" "*"$(Cref.freqmin)-$(Cref.freqmax)Hz", title="Clockshift with $(clockshift*1e3)ms")
p1 = xlims!(-60, 60)
p1 = plot!(legend = :topright, framestyle = :box)

xaxis!("Lag time [s]")
yaxis!("CC")

p2 = plot([-0.002, 0.002], [-0.002, 0.002], ls=:dot, lc="black", labels="")
p2 = scatter!(collect(ϵ), dvv_mwcs, labels="mwcs no origin", ms=10, mc="blue")
p2 = scatter!(collect(ϵ), dvv_mwcs0,  labels="mwcs through origin", ms=7)
p2 = scatter!(collect(ϵ), dvv_stretch, labels="stretching", ms=7, mc="red")

xlims!(-0.0012, 0.0012)
ylims!(-0.0012, 0.0012)
xaxis!("dv/v synthetic true")
yaxis!("dv/v seismeasurement")
p2 = plot!(legend = :bottomright, framestyle = :box, aspect_ratio=1)
p2 = plot!(size=(700,600))

p = plot(p1, p2, layout = grid(2, 1, heights=[0.3, 0.7]))
savefig(p, "./seismeasurement_validation_clockshift_$(Cref.name)_$(Cref.freqmin)-$(Cref.freqmax)Hz_clockshift_$(clockshift*1e3)ms.png")

"/Users/kokubo/Dropbox/NIED_RESEARCH/new_git_SeisMonitoring_Paper/dev_new_git_v2_SeisMonitoring_Paper/Others/validation_of_SeisMeasurement/seismeasurement_validation_clockshift_BP.EADB-BP.LCCB-11_0.9-1.2Hz_clockshift_-30.0ms.png"

In [12]:
tvec

-100.0:0.05:100.0